In [1]:
#Befehle, die verschiedene Funktionen von Spacy laden. Basis für die Programmierung.
import spacy
from collections import Counter
from spacy import displacy
import pandas as pd
from spacytextblob.spacytextblob import SpacyTextBlob

In [2]:
#Englisches Sprachpaket small und Spacytextblobpipe auf nlp laden
nlp = spacy.load ('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [3]:
#Textdatei wird eingelesen, Zeilenumbrüche werden entfernt
text = open("Shape of you - Ed Sheeran.txt").read()
text = text.replace("\n", " ").strip()

"The club isn't the best place to find a lover So the bar is where I go Me and my friends at the table doing shots Drinking fast and then we talk slow And you come over and start up a conversation with just me And trust me I'll give it a chance now Take my hand stop put Van the Man on the jukebox And then we start to dance and now I'm singing like Girl you know I want your love Your love was handmade for somebody like me Come on now follow my lead I may be crazy don't mind me Say boy let's not talk too much Grab on my waist and put that body on me Come on now follow my lead Come come on now follow my lead I'm in love with the shape of you We push and pull like a magnet do Although my heart is falling too I'm in love with your body And last night you were in my room And now my bedsheets smell like you Every day discovering something brand new I'm in love with your body Oh-I-oh-I-oh-I-oh-I I'm in love with your body Oh-I-oh-I-oh-I-oh-I I'm in love with your body Oh-I-oh-I-oh-I-oh-I I'm i

In [4]:
# Funktion, die den Text normalisiert: Entfernt alle Satz- und Lehrzeichen, wandelt konjugierte Verben in Stammform um.
# Gibt am Ende den normalisierten Text zurück. Funktion kann auf jedem beliebigen Text aufgerufen werden

def normalize_ws (text):
    norm_text = []
    for token in text:
            if not token.is_punct and not token.is_space:
                    norm_text.append(token.lemma_.lower())
    return ' '.join(norm_text)

In [5]:
#Text wird in Tokenisierten NLP Text umgewandelt
doc = nlp(text)

In [6]:
# Das "DOC" wird mit dem Funktionsaufruf "normalize_ws" normalisiert und abgespeichert
normalized_ws_text = normalize_ws(doc)
# Das normalisierte DOC muss wieder in einen NLP Text umgewandelt werden
normalized_ws_doc = nlp(normalized_ws_text)
# Alle Nouns werden aus dem normalisierten Text herausgefiltert:
nouns_ws = [ token.text for token in normalized_ws_doc if token.pos_ == 'NOUN']
# Alle Nouns werden aus dem normalisierten Text herausgefiltert:
verbs_ws = [ token.text for token in normalized_ws_doc if token.pos_ == 'VERB']
# Alle Wörter werden als Tokentext abgespeichert
words_ws = [ token.text for token in normalized_ws_doc]
#Anzahl der Token umgewandelt in String und dann in Token
number = len(nlp(normalized_ws_doc)) 
number = str(number)
number = nlp(number)

In [7]:
# Ausgabe zur Kontrolle
#print(nouns_ws)
#print(verbs_ws)
#print(words_ws)
#print (doc)
#print (normalized_ws_doc)

In [8]:
# Berechnung der 10 am häufigsten vorkommenden Nouns
ten_nouns = Counter(nouns_ws).most_common(10)
# Berechnung der 10 am häufigsten vorkommenden Verben
ten_verbs = Counter(verbs_ws).most_common(10)
#Berechnung der 10 am häufigsten vorkommenden Wörter
ten_words = Counter(words_ws).most_common(10)

In [9]:
#Arrays werden initialisiert
song_sent_score = []
song_sent_label = []
song_sent_subjectivity = []
total_pos = []
total_neg = []
positive_words = []
negative_words = []

#Sentiment und Subjectivity wird dem Lied zugeordnet
sentiment = doc._.blob.polarity
sentiment = round(sentiment,2)
subjectivity = doc._.blob.subjectivity
subjectivity = round(subjectivity,2)

#Sentimentscore wird verwendet um das Lied als positives oder negatives einzustufen
if sentiment > 0:
  sent_label = "Positive"
else:
  sent_label = "Negative"

#Resultate werden den entsprechenden Kategorien zugeordnet
song_sent_label.append(sent_label)
song_sent_score.append(sentiment)
song_sent_subjectivity.append(subjectivity)

#Einzelne Tupel (Token + Sentiment) werden als positives oder negatives Wort geteilt
for x in doc._.blob.sentiment_assessments.assessments:
  if x[1] > 0:
    positive_words.append(x[0][0])
  elif x[1] < 0:
    negative_words.append(x[0][0])
  else:
    pass

#Resultate werden den entsprechenden Kategorien zugeordnet
total_pos.append(', '.join(set(positive_words)))
total_neg.append(', '.join(set(negative_words)))

In [10]:
#Fehlende Arrays werden erstellt
total_nouns = []
total_verbs = []
total_words = []
total_amount = []

#Listen werden mit Panda als Basisdatei eingelesen
liste = pd.read_csv("liste.csv", index_col="Nr.")
liste1 = pd.read_csv("liste1.csv", index_col="Nr.")

#Liste wird mit Daten gefüllt
liste ["Häufigste Nouns"]= ten_nouns
liste ["Häufigste Verben"]= ten_verbs
liste ["Häufigste Wörter"]= ten_words

#Liste 1 wird mit Daten gefüllt
liste1["Anzahl aller Token"] = number
liste1["Sentiment Score"] = song_sent_score
liste1["Sentiment Label"] = song_sent_label
liste1["Sentiment Subjectivity"] = song_sent_subjectivity
liste1["Positive Wörter"] = total_pos
liste1["Negative Wörter"] = total_neg


#Listen werden als CSV gespeichert
liste.to_csv("Ergebnisse\ergebnisse.csv")
liste1.to_csv("Ergebnisse\sentiment.csv")